## Решение задачи бинарной классификации

План практической части:
* решим задачу классификации с помощью логистической регрессии
* изучим пайплайны
* научимся подбирать гиперпараметры моделей

### Логистическая регрессия

y = {-1, 1}

$a(x) = \sigma(<w,x>)$, где $\sigma(z) = \frac{1}{1 + e^{-z}}$

Логистическая регрессия - это линейный классификатор, который кроме классов умеет предсказывать вероятности классов, а именно, $a(x) = P(y = +1 | x)$


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
data = pd.read_csv('bike_buyers_clean.csv')

In [ ]:
data.head()

In [ ]:
data.drop(['ID'],axis=1,inplace=True)

# Обзор данных

In [ ]:
# проверим типы колонок в датасете
data.dtypes

## Кодируем категориальные столбцы

In [ ]:
# data = pd.get_dummies(data, columns=['Marital Status', 'Gender', 'Education', 'Occupation',\
#                                     'Commute Distance', 'Region'], drop_first=True)

data.head()

Оставим в данных только числовые колонки.

In [ ]:
X = data._get_numeric_data()
y = data['Purchased Bike']

X.head()

In [ ]:
y.head()

Проверим сбалансированность классов.

In [ ]:
y.value_counts(normalize=True)

Переведем классы в числа 0 и 1.

In [ ]:
y = (y == 'Yes').astype(int)
y

## Масштабирование числовых признаков

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(X[['Income']])

Важное замечание:
- любое преобразование признаков в ML-задачах нужно обучать только на тренировочных данных

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train) # алгоритм находит параметры: m и sigma

X_train_sc = scaler.transform(X_train) # нормирует: x->(x-m)/sigma
X_test_sc = scaler.transform(X_test)

In [ ]:
X_train_sc

In [ ]:
X_train = pd.DataFrame(X_train_sc, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_test_sc, index=X_test.index, columns=X_test.columns)

X_train.head()

Обучим логистическую регрессию и посмотрим на качество модели.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

Pipeline (пайплайн) - инструмент sklearn, позволяющий объединять в единую структуру несколько шагов по работе с данными. Готовый pipeline ведёт себя как обычная модель, то есть, имеет методы fit и predict.

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler_', StandardScaler()),
    ('model_', LogisticRegression(C=100))
    ])

In [ ]:
# пример работы с пайплайном

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print(accuracy_score(y_test, y_pred))

## Интерпретация результатов

In [ ]:
model.coef_, model.intercept_

In [ ]:
coefs = pd.DataFrame(model.coef_, index=np.arange(len(model.coef_)), columns=X_train.columns)
coefs['Intercept'] = model.intercept_
coefs

## Подбор гиперпараметров модели

In [ ]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression()

cross_val_score(model, X, y, cv=3).mean()

In [ ]:
from sklearn.model_selection import GridSearchCV # поиск по сетке

params = {'C' : [0.0001, 0.001, 0.01, 0.1, 0.3, 0.7, 1, 10, 20, 100, 1000],
          'class_weight' : [None, 'balanced'],
          'penalty' : ['l1', 'l2']} # регуляризация

gs = GridSearchCV(LogisticRegression(solver='liblinear'), params, cv=3, scoring='accuracy')
gs.fit(X, y)

In [ ]:
gs.best_score_

In [ ]:
gs.best_estimator_